# Link Prediction with Twitter Friends data  

Link prediction is a key problem for network-structured data. Examples of link prediction include predicting friendship links among users in a social network, predicting co-authorship links in a citation network, and predicting interactions between genes and proteins in a biological network.  

In this notebook, mostly inspired from chapter 6 of [Graph Machine Learning from Packt Publishing](https://github.com/PacktPublishing/Graph-Machine-Learning), we apply basic Machine Learning tools to data from users followed by the main political leaders in Italy.  

We split the graph into a training and testing graph, and then we solve the link prediction task employing and comparing three approaches:
* node2vec-based method;
* GraphSage algorythm;
* Hand-Crafted features method.

In [63]:
import os 
import numpy as np
import pandas as pd
import networkx as nx
import community 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn import metrics 
from stellargraph import StellarGraph
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification
from tensorflow import keras
from node2vec import Node2Vec
from gensim.models import Word2Vec
from node2vec.edges import HadamardEmbedder

In [2]:
def draw_graph(G, pos_nodes, relevant_nodes=[], node_size=3, relevant_node_size=100):
    """
        Draws a Graph and highlights relevant nodes in it.

    :param G: graph data;
    :param pos_nodes: nodes position in graph (layout);
    :param list relevant_nodes: relevant nodes to highlight in the network, default to an empty list;
    :param float node_size: size of every node in the network, default to 50 but can also be a list;
    :param float relevant_node_size: size of nodes to highlight in the network, defaults to 100 but can also be a list.
    
    """
    node_labels = {}
    for node in G.nodes():
        if node in relevant_nodes:
            #set the node name as the key and the label as its value 
            node_labels[node] = node
    plt.figure(figsize=(12, 9))
    # set the argument 'with labels' to False so you have unlabeled graph
    nx.draw(G, pos_nodes, with_labels=False, node_size=node_size, edge_color='grey')
    # draw relevant nodes
    nx.draw_networkx_nodes(G, pos_nodes, nodelist=relevant_nodes, node_size=relevant_node_size, node_color='r', alpha=0.5)
    # only add labels to the nodes you require
    nx.draw_networkx_labels(G, pos_nodes, node_labels, font_size=16, font_color='r', font_weight='bold')
    pos_attrs = {}
    for node, coords in pos_nodes.items():
        pos_attrs[node] = (coords[0], coords[1] + 0.08)
    
    plt.axis('off')
    axis = plt.gca()
    axis.set_xlim([1.2*x for x in axis.get_xlim()])
    axis.set_ylim([1.2*y for y in axis.get_ylim()])

In [3]:
political_net = pd.read_csv('C:/Users/Dylan/Desktop/Projects/working_with_twitter/data/politicians_network.csv')
political_net = political_net[['source', 'target']]
politicians = list(political_net['source'].unique())
political_net.shape

(28421, 2)

In [4]:
G = nx.from_pandas_edgelist(df=political_net ,source='source',target='target', edge_attr=None, create_using=nx.Graph())
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 24192
Number of edges: 28302
Average degree:   2.3398


Since graph embeddings algoryhtm are computationally espensive, for the sake of learning we will remove from the graph those users with less than 3 connections

In [5]:
remove = [node for node,degree in dict(G.degree()).items() if degree < 3]
G.remove_nodes_from(nodes=remove)
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 816
Number of edges: 3087
Average degree:   7.5662


In [6]:
# draw_graph(G, pos_nodes=nx.kamada_kawai_layout(G), relevant_nodes=politicians, node_size=15, relevant_node_size=100)

## Splitting with Stellargraph

The ***stellargraph*** library provides a tool for splitting data and creating training and test reduced **subgraphs**.  

With the EdgeSplitter class we extract a fraction (p = 10%) of all the edges in G, as well as the same number of negative edges, in order to obtain a reduced graph, graph_test. The train_test_split method also return a lsit of node pairs, samples_test (where each pair corresponds to an existing or not existing edge in the graph) and a list of binary targets (labels_test) of the same length of the samples_test list. Then, from such a reduced graph, we are repeating the operation to obtain another reduced graph (graph_train), as well as the corresponding samples_train, and labels_train lists.

In [7]:
edgeSplitter = EdgeSplitter(G)
graph_test, samples_test, labels_test = edgeSplitter.train_test_split(p=0.1, method='global', seed=42)
edgeSplitter = EdgeSplitter(graph_test, G)
graph_train, samples_train, labels_train = edgeSplitter.train_test_split(p=0.1, method='global', seed=42)

** Sampled 308 positive and 308 negative edges. **
** Sampled 277 positive and 277 negative edges. **


In [8]:
print(nx.info(graph_train))

Name: 
Type: Graph
Number of nodes: 816
Number of edges: 2502
Average degree:   6.1324


We will be comparing three different methods for predicting missing edges:  

* Method 1: **node2vec** will be used to learn a node embedding without supervision. The learned embedding will be used as input for a supervised classification 
algorithm to determine whether the input pair is actually connected;
* Method 2: The graph neural network-based algorithm **GraphSAGE** will be used to jointly learn the embedding and perform the classification task;  
* Method 3: **Hand-crafted features** will be extracted from the graph and used as inputs for a supervised classifier, together with the nodes' IDs


## **node2vec**-based link prediction

We use node2vec to generate node embeddings **without supervision** from the 
training graph. This can be done using the node2vec.

In [9]:
node2vec = Node2Vec(graph_train, dimensions=64, walk_length=30, num_walks=200, workers=4) 

Computing transition probabilities: 100%|██████████| 816/816 [00:04<00:00, 178.48it/s]


In [10]:
model = node2vec.fit()

Then, we use **HadamardEmbedder** for generating an embedding for each pair of 
embedded nodes. Such feature vectors will be used as input to train the classifier:

In [11]:
edges_embs = HadamardEmbedder(keyed_vectors=model.wv)
train_embeddings = [edges_embs[str(x[0]),str(x[1])] for x in samples_train]

Next, we will perform training of our supervised classifier. We will be using the Random Forest Classifier.

In [12]:
rf = RandomForestClassifier(n_estimators=10)
rf.fit(train_embeddings, labels_train)

RandomForestClassifier(n_estimators=10)

Apply the trained model for creating the embedding of the test set

In [13]:
edges_embs = HadamardEmbedder(keyed_vectors=model.wv) 
test_embeddings = [edges_embs[str(x[0]),str(x[1])] for x in samples_test]

Predict on the test set using our trained model

In [14]:
y_pred = rf.predict(test_embeddings) 
print('Precision:', metrics.precision_score(labels_test, y_pred)) 
print('Recall:', metrics.recall_score(labels_test, y_pred)) 
print('F1-Score:', metrics.f1_score(labels_test, y_pred))

Precision: 0.9139784946236559
Recall: 0.827922077922078
F1-Score: 0.8688245315161841


## GRAPHSAGE link prediction

Next, we will use GraphSAGE for learning node embeddings and classifying edges. We will build a two-layer **GraphSAGE architecture** that, given labeled pairs of nodes, outputs a pair of node embeddings. Then, a fully connected neural network will be used to process these embeddings and produce link predictions. Notice that the GraphSAGE model and the fully connected network will be concatenated and trained end to end so that the embeddings learning stage is influenced by the predictions.  
### Featurless Approach
GraphSAGE needs node descriptors (features). Such features may or may not be available in your dataset. Let's begin our analysis by not considering available node features. In this case, a common approach is to assign to each node a one-hot feature vector of length |V| (the number of nodes in the graph), where only the cell corresponding to the given node is 1, while the remaining cells are 0

In [54]:
eye = np.eye(graph_train.number_of_nodes(), dtype=int)
i=0
fake_features = {}
for n in G.nodes():
    fake_features[n] = eye[i]
    i = i+1
nx.set_node_attributes(graph_train, fake_features, "fake")

eye = np.eye(graph_test.number_of_nodes(), dtype=int)
i=0
fake_features = {}
for n in G.nodes():
    fake_features[n] = eye[i]
    i = i+1
nx.set_node_attributes(graph_test, fake_features, "fake")

This process is repeated for both the training and the testing graph:
1. We created an identity matrix of size |V|. Each row of the matrix is the one-hot vector we need for each node in the graph.  
2. Then, we created a dictionary where, for each nodeID (used as the key), we assign the corresponding row of the previously created identity matrix.  
3. Finally, the dictionary was passed to the networkx set_node_attributes function to assign the "fake" features to each node in the networkx graph.  

The next step will be defining the generator that will be used to feed the model. We will be using the stellargraph **GraphSAGELinkGenerator** for this, which provides the model with pairs of nodes as input

In [55]:
batch_size = 64 # batch size is the number of inputs for minibatch
num_samples = [4, 4] # number of first and second-hop neighbor samples that GraphSage should consider 

# convert graph_train and graph_test for stellargraph
sg_graph_train = StellarGraph.from_networkx(graph_train, node_features="fake")
sg_graph_test = StellarGraph.from_networkx(graph_test, node_features="fake")
train_gen = GraphSAGELinkGenerator(sg_graph_train, batch_size, num_samples)
train_flow = train_gen.flow(samples_train, labels_train, shuffle=True, seed=24)
test_gen = GraphSAGELinkGenerator(sg_graph_test, batch_size, num_samples)
test_flow = test_gen.flow(samples_test, labels_test, seed=24)

### Model creation

Create a GraphSAGE model with two hidden layers of size 20, each with a bias term and a dropout layer for reducing overfitting.  

Then, the output of the GraphSAGE part of the module is concatenated with a **link_classification layer** that takes pairs of node embeddings (output of GraphSAGE), uses binary operators (*inner product*; ip in our case) to produce edge embeddings, and finally passes them through a fully connected neural network for classification.  

The model is optimized via the Adam optimizer (learning rate = 1e-3) using the mean squared error as a loss function

In [56]:
layer_sizes = [20, 20]
graphsage = GraphSAGE(layer_sizes=layer_sizes, generator=train_gen, bias=True, dropout=0.3)
x_inp, x_out = graphsage.in_out_tensors()
# define the link classifier
prediction = link_classification(output_dim=1, output_act="sigmoid", edge_embedding_method="ip")(x_out)
model = keras.Model(inputs=x_inp, outputs=prediction)
model.compile( optimizer=keras.optimizers.Adam(lr=1e-3), loss=keras.losses.mse, metrics=["acc"],)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


Training the model

In [57]:
epochs = 10
history = model.fit(train_flow, epochs=epochs, validation_data=test_flow)

Epoch 1/10
9/9 [==============================] - 3s 294ms/step - loss: 0.2563 - acc: 0.5235 - val_loss: 0.2500 - val_acc: 0.5422
Epoch 2/10
9/9 [==============================] - 1s 146ms/step - loss: 0.2460 - acc: 0.5614 - val_loss: 0.2457 - val_acc: 0.5519
Epoch 3/10
9/9 [==============================] - 1s 143ms/step - loss: 0.2461 - acc: 0.5740 - val_loss: 0.2365 - val_acc: 0.5877
Epoch 4/10
9/9 [==============================] - 1s 126ms/step - loss: 0.2364 - acc: 0.6011 - val_loss: 0.2355 - val_acc: 0.5893
Epoch 5/10
9/9 [==============================] - 1s 135ms/step - loss: 0.2299 - acc: 0.6480 - val_loss: 0.2358 - val_acc: 0.6104
Epoch 6/10
9/9 [==============================] - 1s 132ms/step - loss: 0.2296 - acc: 0.6155 - val_loss: 0.2336 - val_acc: 0.6169
Epoch 7/10
9/9 [==============================] - 1s 124ms/step - loss: 0.2303 - acc: 0.6336 - val_loss: 0.2311 - val_acc: 0.6169
Epoch 8/10
9/9 [==============================] - 1s 121ms/step - loss: 0.2309 - acc: 0.63

Performance Metrics are lower than in the node2vec approach..

In [58]:
y_pred = np.round(model.predict(train_flow)).flatten()
print('Precision:', metrics.precision_score(labels_train, y_pred)) 
print('Recall:', metrics.recall_score(labels_train, y_pred)) 
print('F1-Score:', metrics.f1_score(labels_train, y_pred)) 

Precision: 0.4839506172839506
Recall: 0.7075812274368231
F1-Score: 0.5747800586510264


## GraphSage with Node Features

The process of extracting node features for the combined ego network is quite verbose.  
This is because each ego network is described using several files, as well as all the feature names and values.

In [59]:
feat_file_name = "feature_map.txt"
feature_index = {}  # numeric index to name
inverted_feature_index = {} # name to numeric index
network = nx.Graph()

def parse_featname_line(line):
  """ used to parse each line of the files containing feature names """
  line = line[(line.find(' '))+1:]  # chop first field
  split = line.split(';')
  name = ';'.join(split[:-1]) # feature name
  index = int(split[-1].split(" ")[-1]) #feature index
  return index, name


def load_features():
  """ 
  parse each ego-network and creates two dictionaries:
      - feature_index: maps numeric indices to names
      - inverted_feature_index: maps names to numeric indices
  """
  import glob
  feat_file_name = 'tmp.txt'
  # may need to build the index first
  if not os.path.exists(feat_file_name):
      feat_index = {}
      # build the index from data/*.featnames files
      featname_files = glob.iglob("facebook/*.featnames")
      for featname_file_name in featname_files:
          featname_file = open(featname_file_name, 'r')
          for line in featname_file:
              # example line:
              # 0 birthday;anonymized feature 376
              index, name = parse_featname_line(line)
              feat_index[index] = name
          featname_file.close()
      keys = feat_index.keys()
      keys = sorted(keys)
      out = open(feat_file_name,'w')
      for key in keys:
          out.write("%d %s\n" % (key, feat_index[key]))
      out.close()

  index_file = open(feat_file_name,'r')
  for line in index_file:
      split = line.strip().split(' ')
      key = int(split[0])
      val = split[1]
      feature_index[key] = val
  index_file.close()

  for key in feature_index.keys():
      val = feature_index[key]
      inverted_feature_index[val] = key


def parse_nodes(network, ego_nodes):
  """
  for each nodes in the network assign the corresponding features 
  previously loaded using the load_features function
  """
  # parse each node
  for node_id in ego_nodes:
      featname_file = open(f'facebook/{node_id}.featnames','r')
      feat_file     = open(f'facebook/{node_id}.feat','r')
      egofeat_file  = open(f'facebook/{node_id}.egofeat','r')
      edge_file     = open(f'facebook/{node_id}.edges','r')

      ego_features = [int(x) for x in egofeat_file.readline().split(' ')]

      # Add ego node features
      network.nodes[node_id]['features'] = np.zeros(len(feature_index))
      
      # parse ego node
      i = 0
      for line in featname_file:
          key, val = parse_featname_line(line)
          # Update feature value if necessary
          if ego_features[i] + 1 > network.nodes[node_id]['features'][key]:
              network.nodes[node_id]['features'][key] = ego_features[i] + 1
          i += 1

      # parse neighboring nodes
      for line in feat_file:
          featname_file.seek(0)
          split = [int(x) for x in line.split(' ')]
          node_id = split[0]
          features = split[1:]

          # Add node features
          network.nodes[node_id]['features'] = np.zeros(len(feature_index))

          i = 0
          for line in featname_file:
              key, val = parse_featname_line(line)
              # Update feature value if necessary
              if features[i] + 1 > network.nodes[node_id]['features'][key]:
                  network.nodes[node_id]['features'][key] = features[i] + 1
              i += 1
          
      featname_file.close()
      feat_file.close()
      egofeat_file.close()
      edge_file.close()

### Hand-Crafted Features for Link Prediction

for each input edge, we will compute a set of metrics that will be given as input to a classifier.  

In this example, for each input edge represented as a pair of nodes (u,v), four metrics will be considered, namely the following

* **Shortest path**: The length of the shortest path between u and v. If u and v are directly connected through an edge, this edge will be removed before computing the shortest path. The value 0 will be used if u is not reachable from v;
* **Jaccard coefficient**: Given a pair of nodes (u,v), it is defined as the intersection over a union of the set of neighbors of u and v;
* **u centrality**: The degree centrality computed for node v;
* **v centrality**: The degree centrality computed for node u;
* **u community**: The community ID assigned to node u using the Louvain heuristic;
* **v community**: The community ID assigned to node v using the Louvain heuristic.


In [64]:
def get_shortest_path(G,u,v):
  """ 
  return the shortest path length between u,v in the graph
  without the edge (u,v) 
  """
  removed = False
  if G.has_edge(u,v):
    removed = True
    G.remove_edge(u,v) # temporary remove edge
  try:
    sp = len(nx.shortest_path(G, u, v))
  except:
    sp = 0
  if removed:
    G.add_edge(u,v) # add back the edge if it was removed
  return sp

def get_hc_features(G, samples_edges, labels):
  # precompute metrics
  centralities = nx.degree_centrality(G)
  parts = community.best_partition(G)
  feats = []
  for (u,v),l in zip(samples_edges, labels):
    shortest_path = get_shortest_path(G, u, v)
    j_coefficient = next(nx.jaccard_coefficient(G, ebunch=[(u, v)]))[-1]
    u_centrality = centralities[u]
    v_centrality = centralities[v]
    u_community = parts.get(u)
    v_community = parts.get(v)
    # add the feature vector
    feats += [[shortest_path, j_coefficient, u_centrality, v_centrality]]
  return feats

In [65]:
feat_train = get_hc_features(graph_train, samples_train, labels_train)
feat_test = get_hc_features(graph_test, samples_test, labels_test)

Now, let's train again a RF Classifier

In [68]:
rf = RandomForestClassifier(n_estimators=10) 
rf.fit(feat_train, labels_train)

RandomForestClassifier(n_estimators=10)

In [69]:
y_pred = rf.predict(feat_test)
print('Precision:', metrics.precision_score(labels_test, y_pred))
print('Recall:', metrics.recall_score(labels_test, y_pred)) 
print('F1-Score:', metrics.f1_score(labels_test, y_pred))

Precision: 0.9967532467532467
Recall: 0.9967532467532467
F1-Score: 0.9967532467532467


**WOW**!!!

Summing up the results, the node2vec-based method is already able to achieve a high level of performance without supervision and per-node information. Such high results might be related to the particular structure of the combined ego network. Due to the high sub-modularity of the network (since it is composed of several ego networks), predicting whether two users will be connected or not might be highly related to the way the two candidate nodes are connected inside the network. For example, there might be a systematic situation in which two users, both connected to several users in the same 
ego network, have a high chance of being connected as well. On the other hand, two users belonging to different ego networks, or very far from each other, are likely to not be connected, making the prediction task easier. This is also confirmed by the high results 
achieved using the shallow method.